In [1]:
pip install websockets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 3.3 MB/s eta 0:00:00


In [6]:
#server side implementation

import asyncio
import websockets
import json

# Define constants
GRID_SIZE = 5
INITIAL_POSITIONS = {
    "A": ["A-P1", "A-H1", "A-P2", "A-H2", "A-P3"],
    "B": ["B-P1", "B-H1", "B-P2", "B-H2", "B-P3"]
}

# Game state
class GameState:
    def __init__(self):
        self.board = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.turn = "A"
        self.players = {"A": None, "B": None}

    def reset_game(self):
        self.__init__()

    def initialize_game(self):
        for i, char in enumerate(INITIAL_POSITIONS["A"]):
            self.board[0][i] = char
        for i, char in enumerate(INITIAL_POSITIONS["B"]):
            self.board[4][i] = char

    def make_move(self, player, move):
        # Implement move logic based on character types
        pass

    def get_board(self):
        return self.board

    def switch_turn(self):
        self.turn = "B" if self.turn == "A" else "A"

game_state = GameState()

async def notify_state(websocket):
    if websocket:
        await websocket.send(json.dumps({"type": "update", "board": game_state.get_board(), "turn": game_state.turn}))

async def handle_move(websocket, player, move):
    valid_move = game_state.make_move(player, move)
    if valid_move:
        game_state.switch_turn()
        await notify_state(game_state.players["A"])
        await notify_state(game_state.players["B"])
    else:
        await websocket.send(json.dumps({"type": "invalid", "message": "Invalid move"}))

async def handler(websocket, path):
    player = None
    if game_state.players["A"] is None:
        player = "A"
        game_state.players["A"] = websocket
    elif game_state.players["B"] is None:
        player = "B"
        game_state.players["B"] = websocket

    if player is None:
        await websocket.send(json.dumps({"type": "error", "message": "Game is full"}))
        return

    await websocket.send(json.dumps({"type": "welcome", "player": player}))
    game_state.initialize_game()
    await notify_state(websocket)

    try:
        async for message in websocket:
            data = json.loads(message)
            if data["type"] == "move":
                await handle_move(websocket, player, data["move"])
    finally:
        game_state.players[player] = None

start_server = websockets.serve(handler, "localhost", 8765)

asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()


RuntimeError: This event loop is already running

In [12]:
#web client implementation
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chess-Like Game</title>
    <style>
        #board {
            display: grid;
            grid-template-columns: repeat(5, 50px);
            grid-template-rows: repeat(5, 50px);
            gap: 5px;
        }
        .cell {
            width: 50px;
            height: 50px;
            border: 1px solid black;
            text-align: center;
            line-height: 50px;
        }
    </style>
</head>
<body>
    <h1>Chess-Like Game</h1>
    <div id="board"></div>
    <div id="status"></div>

    <script>
        const boardElement = document.getElementById("board");
        const statusElement = document.getElementById("status");
        let websocket;
        let player;

        function initializeWebSocket() {
            websocket = new WebSocket("ws://localhost:8765/");
            websocket.onmessage = function(event) {
                const data = JSON.parse(event.data);
                if (data.type === "welcome") {
                    player = data.player;
                    statusElement.textContent = `You are Player ${player}`;
                } else if (data.type === "update") {
                    renderBoard(data.board);
                    statusElement.textContent = `Turn: Player ${data.turn}`;
                } else if (data.type === "invalid") {
                    alert(data.message);
                }
            };
        }

        function renderBoard(board) {
            boardElement.innerHTML = "";
            for (let row of board) {
                for (let cell of row) {
                    const cellElement = document.createElement("div");
                    cellElement.className = "cell";
                    cellElement.textContent = cell;
                    boardElement.appendChild(cellElement);
                }
            }
        }

        initializeWebSocket();
    </script>
</body>
</html>


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 14)

In [3]:
#Game Moves

class GameState:
    def __init__(self):
        self.board = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.turn = "A"
        self.players = {"A": None, "B": None}
        self.positions = {"A": [], "B": []}  # Track positions of all characters

    def reset_game(self):
        self.__init__()

    def initialize_game(self):
        for i, char in enumerate(INITIAL_POSITIONS["A"]):
            self.board[0][i] = char
            self.positions["A"].append((0, i))
        for i, char in enumerate(INITIAL_POSITIONS["B"]):
            self.board[4][i] = char
            self.positions["B"].append((4, i))

    def make_move(self, player, move):
        character_type, direction = move.split(":")
        row, col = self.get_position(player, character_type)

        if character_type.startswith("P"):  # Pawn
            new_row, new_col = self.move_pawn(row, col, direction)
        elif character_type.startswith("H1"):  # Hero1
            new_row, new_col = self.move_hero1(row, col, direction)
        elif character_type.startswith("H2"):  # Hero2
            new_row, new_col = self.move_hero2(row, col, direction)
        else:
            return False  # Invalid character type

        if self.is_valid_move(player, new_row, new_col):
            self.update_board(row, col, new_row, new_col, character_type)
            return True
        return False

    def get_position(self, player, character_type):
        for i, (r, c) in enumerate(self.positions[player]):
            if self.board[r][c] == f"{player}-{character_type}":
                return r, c
        return None, None

    def move_pawn(self, row, col, direction):
        if direction == "L":
            return row, col - 1
        elif direction == "R":
            return row, col + 1
        elif direction == "F":
            return row - 1 if self.turn == "A" else row + 1
        elif direction == "B":
            return row + 1 if self.turn == "A" else row - 1
        return row, col

    def move_hero1(self, row, col, direction):
        if direction == "L":
            return row, col - 2
        elif direction == "R":
            return row, col + 2
        elif direction == "F":
            return row - 2 if self.turn == "A" else row + 2
        elif direction == "B":
            return row + 2 if self.turn == "A" else row - 2
        return row, col

    def move_hero2(self, row, col, direction):
        if direction == "FL":
            return row - 2 if self.turn == "A" else row + 2, col - 2
        elif direction == "FR":
            return row - 2 if self.turn == "A" else row + 2, col + 2
        elif direction == "BL":
            return row + 2 if self.turn == "A" else row - 2, col - 2
        elif direction == "BR":
            return row + 2 if self.turn == "A" else row - 2, col + 2
        return row, col

    def is_valid_move(self, player, new_row, new_col):
        if not (0 <= new_row < GRID_SIZE and 0 <= new_col < GRID_SIZE):
            return False  # Out of bounds
        if self.board[new_row][new_col].startswith(player):
            return False  # Cannot move onto own character
        return True

    def update_board(self, old_row, old_col, new_row, new_col, character_type):
        opponent = "B" if self.turn == "A" else "A"
        # Capture opponent's character if present
        if self.board[new_row][new_col].startswith(opponent):
            self.positions[opponent].remove((new_row, new_col))

        # Move the character
        self.board[new_row][new_col] = f"{self.turn}-{character_type}"
        self.board[old_row][old_col] = ""
        for i, (r, c) in enumerate(self.positions[self.turn]):
            if r == old_row and c == old_col:
                self.positions[self.turn][i] = (new_row, new_col)
                break

    def get_board(self):
        return self.board

    def switch_turn(self):
        self.turn = "B" if self.turn == "A" else "A"


In [ ]:
#Server Side Code
import asyncio
import websockets
import json

GRID_SIZE = 5
INITIAL_POSITIONS = {
    "A": ["P1", "H1", "H2", "H1", "P1"],
    "B": ["P1", "H1", "H2", "H1", "P1"]
}

class GameState:
    def __init__(self):
        self.board = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.turn = "A"
        self.players = {"A": None, "B": None}
        self.positions = {"A": [], "B": []}  # Track positions of all characters

    def reset_game(self):
        self.__init__()

    def initialize_game(self):
        for i, char in enumerate(INITIAL_POSITIONS["A"]):
            self.board[0][i] = f"A-{char}"
            self.positions["A"].append((0, i))
        for i, char in enumerate(INITIAL_POSITIONS["B"]):
            self.board[4][i] = f"B-{char}"
            self.positions["B"].append((4, i))

    def make_move(self, player, move):
        character_type, direction = move.split(":")
        row, col = self.get_position(player, character_type)

        if character_type.startswith("P"):  # Pawn
            new_row, new_col = self.move_pawn(row, col, direction)
        elif character_type.startswith("H1"):  # Hero1
            new_row, new_col = self.move_hero1(row, col, direction)
        elif character_type.startswith("H2"):  # Hero2
            new_row, new_col = self.move_hero2(row, col, direction)
        else:
            return False  # Invalid character type

        if self.is_valid_move(player, new_row, new_col):
            self.update_board(row, col, new_row, new_col, character_type)
            self.switch_turn()
            return True
        return False

    def get_position(self, player, character_type):
        for i, (r, c) in enumerate(self.positions[player]):
            if self.board[r][c] == f"{player}-{character_type}":
                return r, c
        return None, None

    def move_pawn(self, row, col, direction):
        if direction == "L":
            return row, col - 1
        elif direction == "R":
            return row, col + 1
        elif direction == "F":
            return row - 1 if self.turn == "A" else row + 1
        elif direction == "B":
            return row + 1 if self.turn == "A" else row - 1
        return row, col

    def move_hero1(self, row, col, direction):
        if direction == "L":
            return row, col - 2
        elif direction == "R":
            return row, col + 2
        elif direction == "F":
            return row - 2 if self.turn == "A" else row + 2
        elif direction == "B":
            return row + 2 if self.turn == "A" else row - 2
        return row, col

    def move_hero2(self, row, col, direction):
        if direction == "FL":
            return row - 2 if self.turn == "A" else row + 2, col - 2
        elif direction == "FR":
            return row - 2 if self.turn == "A" else row + 2, col + 2
        elif direction == "BL":
            return row + 2 if self.turn == "A" else row - 2, col - 2
        elif direction == "BR":
            return row + 2 if self.turn == "A" else row - 2, col + 2
        return row, col

    def is_valid_move(self, player, new_row, new_col):
        if not (0 <= new_row < GRID_SIZE and 0 <= new_col < GRID_SIZE):
            return False  # Out of bounds
        if self.board[new_row][new_col].startswith(player):
            return False  # Cannot move onto own character
        return True

    def update_board(self, old_row, old_col, new_row, new_col, character_type):
        opponent = "B" if self.turn == "A" else "A"
        # Capture opponent's character if present
        if self.board[new_row][new_col].startswith(opponent):
            self.positions[opponent].remove((new_row, new_col))

        # Move the character
        self.board[new_row][new_col] = f"{self.turn}-{character_type}"
        self.board[old_row][old_col] = ""
        for i, (r, c) in enumerate(self.positions[self.turn]):
            if r == old_row and c == old_col:
                self.positions[self.turn][i] = (new_row, new_col)
                break

    def get_board(self):
        return self.board

    def switch_turn(self):
        self.turn = "B" if self.turn == "A" else "A"

async def handler(websocket, path):
    state = GameState()
    state.initialize_game()
    player = None

    # Assign player A or B
    if state.players["A"] is None:
        state.players["A"] = websocket
        player = "A"
    elif state.players["B"] is None:
        state.players["B"] = websocket
        player = "B"

    await websocket.send(json.dumps({"type": "game_state", "board": state.get_board(), "turn": state.turn}))

    async for message in websocket:
        data = json.loads(message)

        if data["type"] == "move":
            if state.turn == player:
                move = data["move"]
                if state.make_move(player, move):
                    # Broadcast new game state to both players
                    for p in state.players.values():
                        if p:
                            await p.send(json.dumps({"type": "game_state", "board": state.get_board(), "turn": state.turn}))
                else:
                    await websocket.send(json.dumps({"type": "invalid_move", "message": "Invalid move. Try again."}))

        elif data["type"] == "reset":
            state.reset_game()
            state.initialize_game()
            for p in state.players.values():
                if p:
                    await p.send(json.dumps({"type": "game_state", "board": state.get_board(), "turn": state.turn}))

async def main():
    async with websockets.serve(handler, "localhost", 8765):
        await asyncio.Future()  # Run forever

if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
#to run the server
python3 server.py
